# Checking segmentation outputs from Organelle-Segmenter-Plugin

### How to use:
- advance through each block of code sequentially by pressing Shift+Enter
- if a block of code has "#### USER INPUT REQUIRED ###" in it, fill in the blanks below that line before running it

________________
## INPUTS
This section of code blocks loads all of the necessary python packages and functions you will need to run through this notebook

In [1]:
# top level imports
from pathlib import Path
import os, sys
from typing import Optional, Union, Dict, List
import itertools 
import glob

import warnings
import time

import numpy as np
import pandas as pd

import napari

### import local python functions in ../infer_subc
sys.path.append(os.path.abspath((os.path.join(os.getcwd(), '..'))))

from infer_subc.core.file_io import (read_czi_image,
                                        export_inferred_organelle,
                                        import_inferred_organelle,
                                        export_tiff,
                                        list_image_files,
                                        read_tiff_image)



from infer_subc.constants import *
from infer_subc.utils.stats import *
from infer_subc.utils.stats_helpers import *
from infer_subc.utils.stats import _assert_uint16_labels
from infer_subc.core.img import label_uint16


import time
%load_ext autoreload
%autoreload 2

In [2]:
# needed to edit this function for a specific case below, so editing locally
def _import_inferred_organelle(name: str, meta_dict: Dict, out_data_path: Path) -> Union[np.ndarray, None]:
    """
    read inferred organelle from ome.tif file

    Parameters
    ------------
    name: str
        name of organelle.  i.e. nuc, lyso, etc.
    meta_dict:
        dictionary of meta-data (ome) from original file
    out_data_path:
        Path object of directory where tiffs are read from

    Returns
    -------------
    exported file name

    """

    # copy the original file name to meta
    img_name = Path(meta_dict["file_name"])  #
    # add params to metadata

    organelle_fname = f"{img_name.stem}-{name}.tif"

    organelle_path = out_data_path / organelle_fname

    if Path.exists(organelle_path):
        # organelle_obj, _meta_dict = read_ome_image(organelle_path)
        organelle_obj = read_tiff_image(organelle_path)  # .squeeze()
        print(f"loaded  inferred {len(organelle_obj.shape)}D `{name}`  from {out_data_path} ")
        return organelle_obj
    else:
        print(f"`{name}` object not found: {organelle_path}")
        raise FileNotFoundError(f"`{name}` object not found: {organelle_path}")

> #### Opening a Napari window -
> #### A new window should pop up when the line of code below is run

In [3]:
viewer = napari.Viewer()

15-Feb-24 13:18:40 - vispy    - WARNING  - QWindowsWindow::setGeometry: Unable to set geometry 2172x1322-1152-1608 (frame: 2194x1378-1163-1653) on QWidgetWindow/"_QtMainWindowClassWindow" on "\\.\DISPLAY2". Resulting geometry: 2606x1587-1150-1601 (frame: 2628x1643-1161-1646) margins: 11, 45, 11, 11 minimum size: 385x498 MINMAXINFO maxSize=0,0 maxpos=0,0 mintrack=792,1052 maxtrack=0,0)


__________________________
## Quality checking segmentations

#### Use this list to determine the index of the image you would like to look at

In [4]:
#### USER INPUT REQUIRED ###

# Copy and paste the paths to the folders where your data is saved inside the quotation marks below. 
# If you have more than one segmentation data folder, include it in the segmentation_data_2 line. If not, type None wihtout quotation marks
# NOTE: for windows, use "/" 
raw_data = "D:/Experiments (C2-117 - current)/C2-121"
segmentation_data = "D:/Experiments (C2-117 - current)/C2-121/20230921_C2-121_3D-analysis/20230921_C2-121_segmentation"
segmentation_data_2 = None
location_tosave_edited_segmentations = "  "
location_to_save_gooddata = "  "

# Inside of the quotation marks, write the suffix associated to each segmentation file
mask_suffix = "masks_A"
lyso_suffix = "lyso"
mito_suffix = "mito"
golgi_suffix = "golgi"
perox_suffix = "perox"
ER_suffix = "ER"
LD_suffix = "LD"

In [48]:
data_root_path = Path("D:/Experiments (C2-117 - current)/C2-121")

raw_data_path = data_root_path / "C2-121_deconvolution/needs analysis"
raw_file_list = list_image_files(raw_data_path,".tiff")

pd.set_option('display.max_colwidth', None)
pd.DataFrame({"Image Name":raw_file_list}).style.set_properties(**{'text-align':'left'})

,Image Name
0,D:\Experiments (C2-117 - current)\C2-121\C2-121_deconvolution\needs analysis\20230727_C2-121_unconditioned_well 10_cell 5_25nM TG_Linear unmixing_0_cmle.ome.tiff
1,D:\Experiments (C2-117 - current)\C2-121\C2-121_deconvolution\needs analysis\20230727_C2-121_unconditioned_well 11_cell 2_50uM NaAsO_Linear unmixing_0_cmle.ome.tiff
2,D:\Experiments (C2-117 - current)\C2-121\C2-121_deconvolution\needs analysis\20230727_C2-121_unconditioned_well 9_cell 1_50uM NaAsO washout_Linear unmixing_0_cmle.ome.tiff


In [49]:
#### USER INPUT REQUIRED ###

# Utilizing the list above as reference, change this index number (left column in table) to select a specific image
num = 2

In [50]:
raw_img_data, raw_meta_dict = read_czi_image(raw_file_list[num])
name = raw_meta_dict['name']
name[0].split(" :: ")[0]

'20230727_C2-121_unconditioned_well 9_cell 1_50uM NaAsO  washout_Linear unmixing_0_cmle.ome'

In [51]:
seg_data_path = data_root_path / "20230921_C2-121_3D-analysis/20230921_C2-121_segmentation"

mask_seg = import_inferred_organelle("masks_A", raw_meta_dict, seg_data_path)
# lyso_seg = import_inferred_organelle("lyso", raw_meta_dict, seg_data_path)
mito_seg = import_inferred_organelle("mito", raw_meta_dict, seg_data_path)
golgi_seg = import_inferred_organelle("golgi", raw_meta_dict, seg_data_path)
perox_seg = import_inferred_organelle("perox", raw_meta_dict, seg_data_path)
ER_seg = import_inferred_organelle("ER", raw_meta_dict, seg_data_path)
LD_seg = import_inferred_organelle("LD", raw_meta_dict, seg_data_path)

loaded  inferred 4D `masks_A`  from D:\Experiments (C2-117 - current)\C2-121\20230921_C2-121_3D-analysis\20230921_C2-121_segmentation 
loaded  inferred 3D `mito`  from D:\Experiments (C2-117 - current)\C2-121\20230921_C2-121_3D-analysis\20230921_C2-121_segmentation 
loaded  inferred 3D `golgi`  from D:\Experiments (C2-117 - current)\C2-121\20230921_C2-121_3D-analysis\20230921_C2-121_segmentation 
loaded  inferred 3D `perox`  from D:\Experiments (C2-117 - current)\C2-121\20230921_C2-121_3D-analysis\20230921_C2-121_segmentation 
loaded  inferred 3D `ER`  from D:\Experiments (C2-117 - current)\C2-121\20230921_C2-121_3D-analysis\20230921_C2-121_segmentation 
loaded  inferred 3D `LD`  from D:\Experiments (C2-117 - current)\C2-121\20230921_C2-121_3D-analysis\20230921_C2-121_segmentation 


In [52]:
### NEED TO FIX THIS TO INCLUDE AN OPTION TO SELECT WHICH THINGS ARE TO BE SELECTED FROM SECOND FOLDER
# if from a different seg path
seg_data_path = data_root_path / "20230921_C2-121_3D-analysis/20240109_C2-121_segmentation"

# nuc_seg = import_inferred_organelle("20230426_test_nuc", raw_meta_dict, seg_data_path)
# cell_seg = import_inferred_organelle("20230426_test_cell", raw_meta_dict, seg_data_path)
# cyto_seg = import_inferred_organelle("20230426_test_cyto", raw_meta_dict, seg_data_path)
# mask_seg = import_inferred_organelle("masks_A", raw_meta_dict, seg_data_path)
lyso_seg = import_inferred_organelle("lyso", raw_meta_dict, seg_data_path)
# mito_seg = import_inferred_organelle("mito", raw_meta_dict, seg_data_path)
# golgi_seg = import_inferred_organelle("golgi", raw_meta_dict, seg_data_path)
# perox_seg = import_inferred_organelle("perox", raw_meta_dict, seg_data_path)
# ER_seg = import_inferred_organelle("ER", raw_meta_dict, seg_data_path)
# LD_seg = import_inferred_organelle("LD", raw_meta_dict, seg_data_path)

loaded  inferred 3D `lyso`  from D:\Experiments (C2-117 - current)\C2-121\20230921_C2-121_3D-analysis\20240109_C2-121_segmentation 


In [53]:
viewer.layers.clear()
viewer.add_image(raw_img_data[0], name='LD_raw', blending='additive')
viewer.add_image(LD_seg, opacity=0.3, colormap='magenta', blending='additive')
viewer.add_image(raw_img_data[1], name='ER_raw', blending='additive')
viewer.add_image(ER_seg, opacity=0.3, colormap='red', blending='additive')
viewer.add_image(raw_img_data[2], name='GL_raw', blending='additive')
viewer.add_image(golgi_seg, opacity=0.3, colormap='yellow', blending='additive')
viewer.add_image(raw_img_data[3], name='LS_raw', blending='additive')
viewer.add_image(lyso_seg, opacity=0.3, colormap='cyan', blending='additive')
viewer.add_image(raw_img_data[4], name='MT_raw', blending='additive')
viewer.add_image(mito_seg, opacity=0.3, colormap='green', blending='additive')
viewer.add_image(raw_img_data[5], name='PO_raw', blending='additive')
viewer.add_image(perox_seg, opacity=0.3, colormap='bop orange', blending='additive')
viewer.add_image(mask_seg, opacity=0.3, blending='additive')

<Image layer 'mask_seg' at 0x202096771f0>

## STOP: Review all of the segmentations in Napari window before continuing
_______________

In [54]:
# for editing cell mask
viewer.layers.clear()
viewer.add_image(raw_img_data)

<Image layer 'raw_img_data' at 0x201be971000>

selecting 0: raw_img_data:masks_A
btn_run_clicked - 0: {'rescale': True, 'weight_ch0': 1, 'weight_ch1': 4, 'weight_ch2': 1, 'weight_ch3': 1, 'weight_ch4': 0, 'weight_ch5': 2, 'weight_ch6': 0, 'weight_ch7': 0, 'weight_ch8': 0, 'weight_ch9': 0}
selecting 1: Create composite image
btn_run_clicked - 1: {'gauss_sigma': 0.0, 'median_size': 0}
selecting 2: Rescale and smooth image
btn_run_clicked - 2: None
selecting 3: Log transform + Scharr edge detection
btn_run_clicked - 3: {'cutoff_size': 50, 'global_method': 'ave_tri_med', 'local_adjust': 0.05}
btn_run_clicked - 3: {'cutoff_size': 50, 'global_method': 'ave_tri_med', 'local_adjust': 0.05}
selecting 4: Global + local threshold (AICSSeg - MO)
btn_run_clicked - 4: {'hole_max': 30, 'hole_min': 0, 'method': '3D', 'min_size': 10}
selecting 4.1: Remove small holes and objects
btn_run_clicked - 4: {'hole_max': 30, 'hole_min': 0, 'method': '3D', 'min_size': 10}
selecting 5.1: Remove small holes and objects
btn_run_clicked - 5: {'fill_filter_method

__________________
# Saving segmentations for analysis to a common folder

In [55]:
# # for all images that didn't require editing, I am going to separate the nuclei and cell masks and save them to a new file location with the correct organelle segementation files.
# # everything will be saved as a copy

# # GOOD ORIGINALS
nuc_seg = mask_seg[2]
# cell_seg = mask_seg[1]

In [56]:
# # identifying new masks for the images that required editing
seg_data_path = data_root_path / "20230921_C2-121_3D-analysis/20240102_C2-121_segmentation-edits"

# UPDATED
# nuc_seg = _import_inferred_organelle("nuc", raw_meta_dict, seg_data_path)
cell_seg = _import_inferred_organelle("cell", raw_meta_dict, seg_data_path)
# perox_seg = _import_inferred_organelle("perox", raw_meta_dict, seg_data_path)
mito_seg = _import_inferred_organelle("mito", raw_meta_dict, seg_data_path)
ER_seg = _import_inferred_organelle("ER", raw_meta_dict, seg_data_path)

loaded  inferred 3D `cell`  from D:\Experiments (C2-117 - current)\C2-121\20230921_C2-121_3D-analysis\20240102_C2-121_segmentation-edits 
loaded  inferred 3D `mito`  from D:\Experiments (C2-117 - current)\C2-121\20230921_C2-121_3D-analysis\20240102_C2-121_segmentation-edits 
loaded  inferred 3D `ER`  from D:\Experiments (C2-117 - current)\C2-121\20230921_C2-121_3D-analysis\20240102_C2-121_segmentation-edits 


In [57]:
# copying all good segmentations to a new location
path_for_quant = data_root_path / "20230921_C2-121_3D-analysis/C2-121_good-segs"

out_file_n = export_inferred_organelle(cell_seg, "cell", raw_meta_dict, path_for_quant)
out_file_n = export_inferred_organelle(nuc_seg, "nuc", raw_meta_dict, path_for_quant)
out_file_n = export_inferred_organelle(LD_seg, "LD", raw_meta_dict, path_for_quant)
out_file_n = export_inferred_organelle(ER_seg, "ER", raw_meta_dict, path_for_quant)
out_file_n = export_inferred_organelle(golgi_seg, "golgi", raw_meta_dict, path_for_quant)
out_file_n = export_inferred_organelle(lyso_seg, "lyso", raw_meta_dict, path_for_quant)
out_file_n = export_inferred_organelle(mito_seg, "mito", raw_meta_dict, path_for_quant)
out_file_n = export_inferred_organelle(perox_seg, "perox", raw_meta_dict, path_for_quant)

saved file: 20230727_C2-121_unconditioned_well 9_cell 1_50uM NaAsO  washout_Linear unmixing_0_cmle.ome-cell
saved file: 20230727_C2-121_unconditioned_well 9_cell 1_50uM NaAsO  washout_Linear unmixing_0_cmle.ome-nuc
saved file: 20230727_C2-121_unconditioned_well 9_cell 1_50uM NaAsO  washout_Linear unmixing_0_cmle.ome-LD
saved file: 20230727_C2-121_unconditioned_well 9_cell 1_50uM NaAsO  washout_Linear unmixing_0_cmle.ome-ER
saved file: 20230727_C2-121_unconditioned_well 9_cell 1_50uM NaAsO  washout_Linear unmixing_0_cmle.ome-golgi
saved file: 20230727_C2-121_unconditioned_well 9_cell 1_50uM NaAsO  washout_Linear unmixing_0_cmle.ome-lyso
saved file: 20230727_C2-121_unconditioned_well 9_cell 1_50uM NaAsO  washout_Linear unmixing_0_cmle.ome-mito
saved file: 20230727_C2-121_unconditioned_well 9_cell 1_50uM NaAsO  washout_Linear unmixing_0_cmle.ome-perox


__________________________
## Quantifying segmentation

In [2]:
def _find_segmentation_tiff_files(prototype:Union[Path,str],
                                  name_list:List[str], 
                                  seg_path:Union[Path,str],
                                  suffix:Union[str, None]=None) -> Dict:
    """
    Find the matching segmentation files to the raw image file based on the raw image file path.

    Paramters:
    ---------
    prototype:Union[Path,str]
        the file path (as a string) for one raw image file; this file should have matching segmentation 
        output files with the same file name root and different file name ending that match the strings 
        provided in name_list
    name_list:List[str]
        a list of file name endings related to what segmentation is that file
    seg_path:Union[Path,str]
        the path (as a string) to the matching segmentation files.
    suffix:Union[str, None]=None
        any additional text that exists between the file root and the name_list ending
        Ex) Prototype = "C:/Users/Shannon/Documents/Python_Scripts/Infer-subc/raw/a48hrs-Ctrl_9_Unmixing.czi"
            Name of organelle file = a48hrs-Ctrl_9_Unmixing-20230426_test_cell.tiff
            result of .stem = "a48hrs-Ctrl_9_Unmixing"
            organelle/cell area type = "cell"
            suffix = "-20230426_test_"
    
    Returns:
    ----------
    a dictionary of file paths for each image type (raw and all the different segmentations)

    """
    # raw
    prototype = Path(prototype)
    if not prototype.exists():
        print(f"bad prototype. please choose an existing `raw` file as prototype")
        return dict()

    out_files = {"raw":prototype}
    seg_path = Path(seg_path) 

    # raw
    if not seg_path.is_dir():
        print(f"bad path argument. please choose an existing path containing organelle segmentations")
        return out_files

    # segmentations
    for org_n in name_list:
        org_name = Path(seg_path) / f"{prototype.stem}{suffix}{org_n}.tiff"
        if org_name.exists(): 
            out_files[org_n] = org_name
        elif org_name.exists() == False: 
            org_name = Path(seg_path) / f"{prototype.stem}{suffix}{org_n}.tif"
            out_files[org_n] = org_name
        else: 
            print(f"{org_n} .tiff file not found in {seg_path} returning")
            out_files[org_n] = None
    
    return out_files 

In [3]:
# for convex hull errors
warnings.simplefilter("ignore", UserWarning)
warnings.simplefilter("ignore", RuntimeWarning)

def _batch_process_quantification(out_file_name: str,
                                  seg_path: Union[Path,str],
                                  out_path: Union[Path, str], 
                                  raw_path: Union[Path,str], 
                                  raw_file_type: str,
                                  organelle_names: List[str],
                                  organelle_channels: List[int],
                                  region_names: List[str],
                                  masks_file_name: [str],
                                  mask: str,
                                  dist_centering_obj:str, 
                                  dist_num_bins: int,
                                  dist_center_on: bool=False,
                                  dist_keep_center_as_bin: bool=True,
                                  dist_zernike_degrees: Union[int, None]=None,
                                  include_contact_dist: bool = True,
                                  scale:bool=True,
                                  seg_suffix:Union[str, None]=None) -> int :
    """  
    batch process segmentation quantification (morphology, distribution, contacts); this function is currently optimized to process images from one file folder per image type (e.g., raw, segmentation)
    the output csv files are saved to the indicated out_path folder

    Parameters:
    ----------
    out_file_name: str
        the prefix to use when naming the output datatables
    seg_path: Union[Path,str]
        Path or str to the folder that contains the segmentation tiff files
    out_path: Union[Path, str]
        Path or str to the folder that the output datatables will be saved to
    raw_path: Union[Path,str]
        Path or str to the folder that contains the raw image files
    raw_file_type: str
        the file type of the raw data; ex - ".tiff", ".czi"
    organelle_names: List[str]
        a list of all organelle names that will be analyzed; the names should be the same as the suffix used to name each of the tiff segmentation files
        Note: the intensity measurements collect per region (from get_region_morphology_3D function) will only be from channels associated to these organelles 
    organelle_channels: List[int]
        a list of channel indices associated to respective organelle staining in the raw image; the indices should listed in same order in which the respective segmentation name is listed in organelle_names
    region_names: List[str]
        a list of regions, or masks, to measure; the order should correlate to the order of the channels in the "masks" output segmentation file
    masks_file_name: str
        the suffix of the "masks" segmentation file; ex- "masks_B", "masks", etc.
        this function currently does not accept indivial region segmentations 
    mask: str
        the name of the region to use as the mask when measuring the organelles; this should be one of the names listed in regions list; usually this will be the "cell" mask
    dist_centering_obj:str
        the name of the region or object to use as the centering object in the get_XY_distribution function
    dist_num_bins: int
        the number of bins for the get_XY_distribution function
    dist_center_on: bool=False,
        for get_XY_distribution:
        True = distribute the bins from the center of the centering object
        False = distribute the bins from the edge of the centering object
    dist_keep_center_as_bin: bool=True
        for get_XY_distribution:
        True = include the centering object area when creating the bins
        False = do not include the centering object area when creating the bins
    dist_zernike_degrees: Union[int, None]=None
        for get_XY_distribution:
        the number of zernike degrees to include for the zernike shape descriptors; if None, the zernike measurements will not 
        be included in the output
    include_contact_dist:bool=True
        whether to include the distribution of contact sites in get_contact_metrics_3d(); True = include contact distribution
    scale:bool=True
        a tuple that contains the real world dimensions for each dimension in the image (Z, Y, X)
    seg_suffix:Union[str, None]=None
        any additional text that is included in the segmentation tiff files between the file stem and the segmentation suffix
        TODO: this can't be None!!! need to update!!!


    Returns:
    ----------
    count: int
        the number of images processed
        
    """
    start = time.time()
    count = 0

    if isinstance(raw_path, str): raw_path = Path(raw_path)
    if isinstance(seg_path, str): seg_path = Path(seg_path)
    if isinstance(out_path, str): out_path = Path(out_path)
    
    if not Path.exists(out_path):
        Path.mkdir(out_path)
        print(f"making {out_path}")
    
    # reading list of files from the raw path
    img_file_list = list_image_files(raw_path, raw_file_type)

    # list of segmentation files to collect
    segs_to_collect = organelle_names + masks_file_name

    # containers to collect data tabels
    org_tabs = []
    contact_tabs = []
    dist_tabs = []
    region_tabs = []
    for img_f in img_file_list:
        count = count + 1
        filez = _find_segmentation_tiff_files(img_f, segs_to_collect, seg_path, seg_suffix)

        # read in raw file and metadata
        img_data, meta_dict = read_czi_image(filez["raw"])

        # create intensities from raw file as list based on the channel order provided
        intensities = [img_data[ch] for ch in organelle_channels]

        # define the scale
        if scale is True:
            scale_tup = meta_dict['scale']
        else:
            scale_tup = None

        # load regions as a list based on order in list (should match order in "masks" file)
        # masks = read_tiff_image(filez[masks_file_name]) 
        # regions = [masks[r] for r, region in enumerate(region_names)]
        regions= [read_tiff_image(filez[masks_file_name[0]]), read_tiff_image(filez[masks_file_name[1]])]

        # store organelle images as list
        organelles = [read_tiff_image(filez[org]) for org in organelle_names]

        org_metrics, contact_metrics, dist_metrics, region_metrics = make_all_metrics_tables(source_file=img_f,
                                                                                             list_obj_names=organelle_names,
                                                                                             list_obj_segs=organelles,
                                                                                             list_intensity_img=intensities, 
                                                                                             list_region_names=region_names,
                                                                                             list_region_segs=regions, 
                                                                                             mask=mask,
                                                                                             dist_centering_obj=dist_centering_obj,
                                                                                             dist_num_bins=dist_num_bins,
                                                                                             dist_center_on=dist_center_on,
                                                                                             dist_keep_center_as_bin=dist_keep_center_as_bin,
                                                                                             dist_zernike_degrees=dist_zernike_degrees,
                                                                                             scale=scale_tup,
                                                                                             include_contact_dist=include_contact_dist)

        org_tabs.append(org_metrics)
        contact_tabs.append(contact_metrics)
        dist_tabs.append(dist_metrics)
        region_tabs.append(region_metrics)
        end2 = time.time()
        print(f"Completed processing for {count} images in {(end2-start)/60} mins.")

    final_org = pd.concat(org_tabs, ignore_index=True)
    final_contact = pd.concat(contact_tabs, ignore_index=True)
    final_dist = pd.concat(dist_tabs, ignore_index=True)
    final_region = pd.concat(region_tabs, ignore_index=True)

    org_csv_path = out_path / f"{out_file_name}organelles.csv"
    final_org.to_csv(org_csv_path)

    contact_csv_path = out_path / f"{out_file_name}contacts.csv"
    final_contact.to_csv(contact_csv_path)

    dist_csv_path = out_path / f"{out_file_name}distributions.csv"
    final_dist.to_csv(dist_csv_path)

    region_csv_path = out_path / f"{out_file_name}regions.csv"
    final_region.to_csv(region_csv_path)

    end = time.time()
    print(f"Quantification for {count} files is COMPLETE! Files saved to '{out_path}'.")
    print(f"It took {(end - start)/60} minutes to quantify these files.")
    return count

In [4]:
seg=_batch_process_quantification(out_file_name= "20240118_C2-121_quant",
                                  seg_path="D:/Experiments (C2-117 - current)/C2-121/20230921_C2-121_3D-analysis/C2-121_good-segs",
                                  out_path="D:/Experiments (C2-117 - current)/C2-121/20230921_C2-121_3D-analysis/20240221_C2-121_quantification_10per", 
                                  raw_path="D:/Experiments (C2-117 - current)/C2-121/C2-121_deconvolution",
                                  raw_file_type = ".tiff",
                                  organelle_names = ['LD', 'ER', 'golgi', 'lyso', 'mito', 'perox'],
                                  organelle_channels= [0,1,2,3,4,5],
                                  region_names= ['nuc', 'cell'],
                                  masks_file_name= ['nuc', 'cell'],
                                  mask= 'cell',
                                  dist_centering_obj='nuc', 
                                  dist_num_bins=5,
                                  dist_center_on=False,
                                  dist_keep_center_as_bin=True,
                                  dist_zernike_degrees=None,
                                  include_contact_dist= True,
                                  scale=True,
                                  seg_suffix="-")

WTF!!  how did we have missing labels?
WTF!!  how did we have missing labels?
WTF!!  how did we have missing labels?
WTF!!  how did we have missing labels?
WTF!!  how did we have missing labels?
WTF!!  how did we have missing labels?
WTF!!  how did we have missing labels?
WTF!!  how did we have missing labels?
WTF!!  how did we have missing labels?
WTF!!  how did we have missing labels?
WTF!!  how did we have missing labels?
WTF!!  how did we have missing labels?
WTF!!  how did we have missing labels?
WTF!!  how did we have missing labels?
WTF!!  how did we have missing labels?
WTF!!  how did we have missing labels?
WTF!!  how did we have missing labels?
WTF!!  how did we have missing labels?
WTF!!  how did we have missing labels?
WTF!!  how did we have missing labels?
WTF!!  how did we have missing labels?
It took 12.85111653804779 minutes to quantify one image.
Completed processing for 1 images in 13.633693202336628 mins.
WTF!!  how did we have missing labels?
WTF!!  how did we have 